### Nota
Este código importa todos los archivos .csv que esten en la carpeta VentaVehiculosMexico y los agrupa en un solo DF, contiene los datos de Venta de Vehículos en México del 2005-2021. 

In [154]:
# Importing dependencies
import pandas as pd
import glob
import geojson
import geopandas as gpd

In [166]:
# Exports CSV
path1 = "VehiculosExportados" # use your path
all_files1 = glob.glob(path1 + "/*.csv")
li1 = []
for filename in all_files1:
    df = pd.read_csv(filename, index_col=None, header=0)
    li1.append(df)
frame1 = pd.concat(li1, axis=0, ignore_index=True)
countriesTable = pd.read_csv("ExportsMap/tc_pais_destino.csv")
exportsDF = pd.merge(frame1, countriesTable, how="left", left_on="ID_PAIS_DESTINO", right_on="ID_PAIS_DESTINO")
brandTotals = exportsDF.groupby("MARCA")["UNI_VEH"].sum().sort_values(ascending=False)
top = brandTotals[0:7]
years = list(range(2006, 2021))
topExportsDF = exportsDF.loc[exportsDF["MARCA"].isin(top.keys())]
yearsTopExportsDF = topExportsDF.loc[topExportsDF["ANIO"].isin(years)]
exportsCountryYear= yearsTopExportsDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
exportsCountryYear.to_csv("ExportsMap/brandexports.csv", index=True)

# Sales CSV
path2 = "VentaVehiculosMexico" # use your path
all_files2 = glob.glob(path2 + "/*.csv")
li2 = []
for filename in all_files2:
    df = pd.read_csv(filename, index_col=None, header=0)
    li2.append(df)
frame2 = pd.concat(li2, axis=0, ignore_index=True)
salesDF = pd.merge(frame2, countriesTable, how="left", left_on="ID_PAIS_ORIGEN", right_on="ID_PAIS_DESTINO")
topSalesDF = salesDF.loc[salesDF["MARCA"].isin(top.keys())]
yearsTopSalesDF = topSalesDF.loc[topSalesDF["ANIO"].isin(years)]
salesCountryYear = yearsTopSalesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
salesCountryYear.to_csv("ExportsMap/brandsales.csv", index=True)

# Balance CSV
balanceCountryYear = exportsCountryYear - salesCountryYear
balanceCountryYear.to_csv("ExportsMap/brandbalance.csv", index=True)